In [1]:
import random
import numpy as np
import itertools
import copy

## Define a linear function for f

In [45]:
n = 10
t = 300
x = np.random.uniform(0,1, n)
a = np.random.uniform(-1,1, n)

In [46]:
x

array([0.39258359, 0.17741823, 0.59250329, 0.54281811, 0.33530203,
       0.62242382, 0.31806745, 0.12839349, 0.88517413, 0.24545588])

In [47]:
a

array([ 0.22190737, -0.70470618, -0.28948519,  0.35585917,  0.44527886,
       -0.74862312, -0.30095507, -0.87786712, -0.1669726 , -0.95652113])

In [48]:
def f_linear(x):   
    return np.dot(a,x.T)

f_linear(x)

-0.9239426147152634

## Approximation for Multi-Linear Extension

In [49]:
x_sample = 0

def F_unfixed(x, f, t, x_sample): 
    # Here, take x_sample as a parameter for the ease of 
    # using both the F and F_fixed functions in this framework

    sum_R = 0

    for i in range(t):

        x_sample = np.random.uniform(0,1, x.shape)

        R_t = x >= x_sample

        sum_R = sum_R + f(R_t)

    return sum_R/t

# In Function F, re-sample from x every time we run the function
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))

-0.7980966155975172
-0.99250439092805
-0.9334419022178256


In [50]:
# Sample from x (stored in x_sample) once and for all  
# x_sample defined here will be used as a parameter for the code below

def get_x_sample(t): 
    x_sample = []
    for i in range(t):
        x_sample.append(np.random.uniform(0,1, x.shape))
        
    return x_sample

In [51]:
x_sample = get_x_sample(t)

def F_fixed(x, f, t, x_sample):
    # Here, we do not re-sample from x every time we run the function
    # Instead, we take a collection of x samples and compute the estimated expectation
    
    sum_R = 0
    for i in range(t):
        R_t = x >= x_sample[i]
        sum_R = sum_R + f(R_t)
    return sum_R/t
                 
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))

-1.0633340141700303
-1.0633340141700303
-1.0633340141700303


## Function to calculate gradients

In [81]:
def get_gradient_F(F,x,f,t,n,x_sample):
    # A vectorized function to get gradients
    
    x_new_1 = x*np.ones((n, n))
    x_new_0 = x*np.ones((n, n))

    np.fill_diagonal(x_new_1, 1)
    np.fill_diagonal(x_new_0, 0)
    
    return F(x_new_1, f, t, x_sample) - F(x_new_0, f, t, x_sample)

print(get_gradient_F(F_fixed,x,f_linear,t,n,x_sample))

[-0.73569541  0.62903284  0.26188165  0.63377371  0.44822793  0.70714339
 -0.16722055 -0.33568168 -0.29152329  0.56592234]


## Gradient ascent

In [211]:
def gradient_ascent(F, x, f, alpha, t, epsilon, n, n_step_max, grad_decay = False, input_x_sample = None):  
    
    if input_x_sample == None:
        x_sample = get_x_sample(t)
    else:
        x_sample = input_x_sample
    
    x = copy.deepcopy(x)
    x_init = copy.deepcopy(x)
    sum_init = F(x, f, t, x_sample)
    
    # key values to be used
    sum_update = 0
    step = 0
    
    sum_temp = copy.deepcopy(sum_init)
    
    # start updating the parameters x with iterative gradients
    while (np.abs(sum_temp - sum_update) > epsilon) & (step < n_step_max) :    
#     while (np.abs(sum_temp - sum_update) > epsilon) & (sum_update > sum_temp) & (step < n_step_max) :
  
        step += 1
        sum_temp = F(x, f, t, x_sample)
        x_temp = x
        
        grad = get_gradient_F(F,x,f,t,n, x_sample)
        
        x = x + alpha*grad
        x = np.maximum(np.minimum(x,1),0)
        
        sum_update = F(x, f, t, x_sample)
        
        if grad_decay == True:
            alpha *= (1/step)**(1/5)
            
        
    print("\n",
          "Iteration: ", step, "\n\n" , 
          "Gradient:  ", grad, "\n\n",
          "x updated: ", x, "\n",
          "Updated  value: ", sum_update,"\n\n", 
          "x second last: ", x_temp, "\n",
          "Second last value: ", sum_temp, "\n\n")
    
        
    
    return step,sum_update, x, x_temp, x_sample

# Testing with a linear function

In [212]:
###
# parameters
###

# coefficients for the linear function
a = np.random.uniform(-1,1, n)

# parameters fo the gradient ascent function
alpha = 0.01
epsilon = 10**(-8)
n_step_max = 5000

# number of iterations for estimating the expectation
t = 300

# number of products in the assortment
n = 10

# initialize the x vector
x_initial = np.random.uniform(0,1, n)
print(x_initial)

[0.69744234 0.39981218 0.93486723 0.77825023 0.6807701  0.12214443
 0.53588094 0.46024644 0.61665985 0.9770157 ]


### Testing for different values of a

In [213]:
# Use function F_fixed: sampling for estimating expectation is only done once and for all
print(a)
# x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)

[-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171]

 Iteration:  112 

 Gradient:   [-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171] 

 x updated:  [0.12048984 1.         0.6354695  1.         0.38639506 0.21471746
 1.         0.         1.         1.        ] 
 Updated  value:  2.551763805394968 

 x second last:  [0.1256412  1.         0.63814269 1.         0.38902341 0.21389091
 1.         0.         1.         1.        ] 
 Second last value:  2.551763805394968 




#### Check that the last two iterations actually converged

In [214]:
x_previous = x_temp_
print("x (2nd last iter): ", x_previous)
print("Value of F (2nd last iter): ", F_fixed(x_previous, f_linear, t, x_sample_), "\n")

x_updated  = x_
print("x (last iter): ", x_updated)
print("Value of F (last iter): ", F_fixed(x_updated, f_linear, t, x_sample_))

x (2nd last iter):  [0.1256412  1.         0.63814269 1.         0.38902341 0.21389091
 1.         0.         1.         1.        ]
Value of F (2nd last iter):  2.551763805394968 

x (last iter):  [0.12048984 1.         0.6354695  1.         0.38639506 0.21471746
 1.         0.         1.         1.        ]
Value of F (last iter):  2.551763805394968


#### If we manually converge probs to 0 or 1...

In [215]:
x_previous_modified = copy.deepcopy(x_previous)
x_previous_modified[0] = 0
x_previous_modified[6] = 1

print(x_previous_modified)
F_fixed(x_previous_modified, f_linear, t, x_sample_) 

[0.         1.         0.63814269 1.         0.38902341 0.21389091
 1.         0.         1.         1.        ]


2.601560301488282

#### The convergence will be actually much faster if we use decaying gradient here
Due to the fact that there is no local maximum in the linear function.
And that decreasing in steps will make the incremental change really small.

In [216]:
step_,sum_update_, x_, x_temp_, x_sample_  = gradient_ascent(F_fixed, x, f_linear, 
                                                             alpha, t, epsilon, n, n_step_max, 
                                                             grad_decay=True, input_x_sample=x_sample)

alpha:  0.01 

alpha:  0.008705505632961241 

alpha:  0.0069882711877157925 

alpha:  0.005296119205244061 

alpha:  0.0038385194963737744 

alpha:  0.0026824615199994182 

alpha:  0.0018176652007284484 

alpha:  0.0011992118057488942 

alpha:  0.000772764910314653 

alpha:  0.0004875816957196081 

alpha:  0.000301834484571944 

alpha:  0.00018362533756728568 

alpha:  0.00010993701395130539 


 Iteration:  13 

 Gradient:   [-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171] 

 x updated:  [0.17152594 0.04480645 0.48227552 0.72706698 0.06837715 1.
 0.41338185 0.95047467 0.02739392 0.93411723] 
 Updated  value:  -0.07578491974910434 

 x second last:  [0.17162053 0.04464906 0.48232461 0.72692453 0.06842541 1.
 0.413271   0.95064865 0.02729769 0.93409917] 
 Second last value:  -0.07578491974910434 




#### Initialization on x matters

In [192]:
x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)

Iteration:  1 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  -0.003686489207583237 
 Previous value:  -0.03329553828997502 
 x:  [0.84210852 0.56696212 0.45468491 0.1738999  0.70253759 0.90460867
 0.37649758 0.42472403 0.22829119 0.70345574] 

Iteration:  2 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.036677052618338986 
 Previous value:  -0.003686489207583237 
 x:  [0.83781132 0.55995761 0.44536035 0.17872347 0.69791874 0.91455439
 0.37906813 0.42786776 0.22392209 0.71041254] 

Iteration:  3 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.07057432321175537 
 Previous value:  0.036677052618338986 
 x:  [0.83351412 0.55295309 0.43603579 0.18354704 0.69329989 0.

Iteration:  26 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.8200868028149738 
 Previous value:  0.7875397315935231 
 x:  [0.73467856 0.39184923 0.22157094 0.29448916 0.58706629 1.
 0.44076134 0.50331735 0.11906362 0.87737565] 

Iteration:  27 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.8340523398608125 
 Previous value:  0.8200868028149738 
 x:  [0.73038136 0.38484472 0.21224638 0.29931273 0.58244744 1.
 0.44333189 0.50646108 0.11469451 0.88433244] 

Iteration:  28 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.8657080035594205 
 Previous value:  0.8340523398608125 
 x:  [0.72608416 0.3778402  0.20292182 0.30413631 0.57782859 1.
 0.44590244 0.50960482 0.

Iteration:  75 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.847539704390585 
 Previous value:  1.8333631066406333 
 x:  [0.52411585 0.04862796 0.         0.53084412 0.36074254 1.
 0.56671831 0.65736025 0.         1.        ] 

Iteration:  76 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.8670337925750546 
 Previous value:  1.847539704390585 
 x:  [0.51981865 0.04162345 0.         0.53566769 0.35612369 1.
 0.56928886 0.66050399 0.         1.        ] 

Iteration:  77 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.8726615771137642 
 Previous value:  1.8670337925750546 
 x:  [0.51552145 0.03461893 0.         0.54049126 0.35150484 1.
 0.57185941 0.66364772 0.  

Iteration:  122 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.2706335781118243 
 Previous value:  2.258536762327536 
 x:  [0.32214753 0.         0.         0.75755194 0.14365649 1.
 0.68753417 0.80511569 0.         1.        ] 

Iteration:  123 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.276505059556079 
 Previous value:  2.2706335781118243 
 x:  [0.31785033 0.         0.         0.76237551 0.13903764 1.
 0.69010472 0.80825943 0.         1.        ] 

Iteration:  124 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.2849334336533955 
 Previous value:  2.276505059556079 
 x:  [0.31355313 0.         0.         0.76719908 0.13441879 1.
 0.69267527 0.81140316 0.

Iteration:  147 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.4798944635369264 
 Previous value:  2.4751391324925662 
 x:  [0.21471757 0.         0.         0.8781412  0.02818519 1.
 0.75179793 0.88370901 0.         1.        ] 

Iteration:  148 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.48928287547127 
 Previous value:  2.4798944635369264 
 x:  [0.21042037 0.         0.         0.88296477 0.02356634 1.
 0.75436848 0.88685274 0.         1.        ] 

Iteration:  149 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.4927272538415397 
 Previous value:  2.48928287547127 
 x:  [0.20612317 0.         0.         0.88778834 0.01894749 1.
 0.75693903 0.88999648 0. 

In [196]:
x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)

Iteration:  1 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.5624456994940936 
 Previous value:  0.5240709814994433 
 x:  [0.07098575 0.54736107 0.74493732 0.73509327 0.69261832 0.85237951
 0.50273469 0.83694893 0.20457393 0.73105408] 

Iteration:  2 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.6279035822456749 
 Previous value:  0.5624456994940936 
 x:  [0.06668855 0.54035655 0.73561276 0.73991684 0.68799946 0.86232523
 0.50530524 0.84009266 0.20020483 0.73801088] 

Iteration:  3 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.6739026699796788 
 Previous value:  0.6279035822456749 
 x:  [0.06239135 0.53335204 0.7262882  0.74474041 0.68338061 0.87227095
 0.5

Iteration:  27 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.4143313118471452 
 Previous value:  1.3895314696868253 
 x:  [0.         0.36524366 0.50249879 0.86050611 0.57252816 1.
 0.569569   0.91868598 0.09097725 0.91193078] 

Iteration:  28 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.4292256669142416 
 Previous value:  1.4143313118471452 
 x:  [0.         0.35823914 0.49317423 0.86532968 0.56790931 1.
 0.57213955 0.92182971 0.08660815 0.91888758] 

Iteration:  29 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.4425504125425679 
 Previous value:  1.4292256669142416 
 x:  [0.         0.35123463 0.48384967 0.87015325 0.56329046 1.
 0.5747101  0.92497345 0.

Iteration:  52 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.9997143713005099 
 Previous value:  1.978698205305894 
 x:  [0.         0.19013077 0.26938482 0.98109537 0.45705686 1.
 0.63383276 0.9972793  0.         1.        ] 

Iteration:  53 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.031612845458047 
 Previous value:  1.9997143713005099 
 x:  [0.         0.18312625 0.26006026 0.98591894 0.45243801 1.
 0.63640331 1.         0.         1.        ] 

Iteration:  54 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.0503995488179574 
 Previous value:  2.031612845458047 
 x:  [0.         0.17612174 0.2507357  0.99074251 0.44781916 1.
 0.63897386 1.         0.   

Iteration:  79 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.4964449426421447 
 Previous value:  2.469099482766655 
 x:  [0.         0.00100884 0.01762173 1.         0.33234785 1.
 0.70323762 1.         0.         1.        ] 

Iteration:  80 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.508165968852068 
 Previous value:  2.4964449426421447 
 x:  [0.         0.         0.00829717 1.         0.327729   1.
 0.70580817 1.         0.         1.        ] 

Iteration:  81 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.5121020536662337 
 Previous value:  2.508165968852068 
 x:  [0.         0.         0.         1.         0.32311015 1.
 0.70837872 1.         0.   

In [197]:
# Use function F: re-sampling is enabled in each iteration

# Note: in this case, the value of the function could decrease as each time the estimated value of F is different 
#       due to differnt samples from x
step_,sum_update_, x_, x_temp_, x_sample_  = gradient_ascent(F_unfixed, x_initial, f_linear, alpha, t, epsilon,
                                                             n, n_step_max, grad_decay=False)

Iteration:  1 
 Gradient:   [-0.43897555 -0.79898384 -1.01586145  0.4290733  -0.54996499  0.96825282
  0.25714482  0.28627991 -0.47880315  0.67392608] 
 Updated  value:  0.5648410798374948 
 Previous value:  0.5486639820034311 
 x:  [0.07089319 0.54637574 0.74410326 0.73456044 0.69173752 0.85211632
 0.50273559 0.836668   0.204155   0.73083654] 

Iteration:  2 
 Gradient:   [-0.31237627 -0.69688134 -0.92438945  0.40383608 -0.50401049  1.0081127
  0.18800046  0.17673746 -0.48258673  0.71112821] 
 Updated  value:  0.5672751188892647 
 Previous value:  0.6252611830170085 
 x:  [0.06776943 0.53940693 0.73485937 0.7385988  0.68669741 0.86219745
 0.5046156  0.83843537 0.19932913 0.73794783] 

Iteration:  3 
 Gradient:   [-0.51682027 -0.66111703 -0.87800354  0.47060617 -0.33226028  1.01235217
  0.16209943  0.29621726 -0.43539734  0.7101899 ] 
 Updated  value:  0.6587402947441607 
 Previous value:  0.5995215497885266 
 x:  [0.06260123 0.53279576 0.72607933 0.74330486 0.68337481 0.87232097
 0.50

Iteration:  25 
 Gradient:   [-0.4148607  -0.7262904  -0.86504479  0.49356929 -0.45474933  0.98441044
  0.24502905  0.23121852 -0.39541469  0.7750851 ] 
 Updated  value:  1.3275550085404189 
 Previous value:  1.3471425633432652 
 x:  [0.         0.37694182 0.52133459 0.84930093 0.58457803 1.
 0.56139818 0.909197   0.09768234 0.89975771] 

Iteration:  26 
 Gradient:   [-0.3481577  -0.82589652 -0.92284798  0.5105583  -0.29938295  0.90618613
  0.23243089  0.32574545 -0.4027257   0.69801396] 
 Updated  value:  1.4431038533513048 
 Previous value:  1.4238304546598421 
 x:  [0.         0.36868286 0.51210611 0.85440651 0.5815842  1.
 0.56372249 0.91245446 0.09365509 0.90673785] 

Iteration:  27 
 Gradient:   [-0.39558016 -0.73124924 -0.97972854  0.43079994 -0.5180384   0.87986848
  0.12106238  0.27563676 -0.44100449  0.71344137] 
 Updated  value:  1.4513997797822114 
 Previous value:  1.4565599504520805 
 x:  [0.         0.36137037 0.50230883 0.85871451 0.57640381 1.
 0.56493311 0.91521082 0.

Iteration:  56 
 Gradient:   [-0.48319137 -0.69950426 -0.941016    0.47521561 -0.40207698  1.03875926
  0.2144828   0.35858209 -0.44572785  0.74142794] 
 Updated  value:  2.1377116530233193 
 Previous value:  2.0153480830855495 
 x:  [0.         0.16062297 0.23250958 0.99772812 0.44197904 1.
 0.63394526 1.         0.         1.        ] 

Iteration:  57 
 Gradient:   [-0.46223673 -0.65427918 -0.93690726  0.48186529 -0.46006061  1.09589758
  0.19754907  0.32322689 -0.41970998  0.69274517] 
 Updated  value:  2.114523194208471 
 Previous value:  2.113749846519203 
 x:  [0.         0.15408018 0.2231405  1.         0.43737843 1.
 0.63592075 1.         0.         1.        ] 

Iteration:  58 
 Gradient:   [-0.40443277 -0.71021152 -0.95345586  0.47310482 -0.44928546  1.00753758
  0.243432    0.24854478 -0.3621027   0.68591968] 
 Updated  value:  2.1440913045264582 
 Previous value:  2.1170469111082615 
 x:  [0.         0.14697807 0.21360594 1.         0.43288558 1.
 0.63835507 1.         0.  

Iteration:  87 
 Gradient:   [-0.39993129 -0.74697469 -0.94135898  0.48834144 -0.45760096  0.99679455
  0.26629273  0.32327638 -0.43846348  0.67890436] 
 Updated  value:  2.5254363613439264 
 Previous value:  2.5155158899470837 
 x:  [0.         0.         0.         1.         0.29964164 1.
 0.71054801 1.         0.         1.        ] 

Iteration:  88 
 Gradient:   [-0.45523768 -0.69240532 -0.92904204  0.4815002  -0.46359891  0.99493389
  0.27707006  0.3164487  -0.42579837  0.71042017] 
 Updated  value:  2.5379273830425926 
 Previous value:  2.531782477737064 
 x:  [0.         0.         0.         1.         0.29500565 1.
 0.71331871 1.         0.         1.        ] 

Iteration:  89 
 Gradient:   [-0.42235634 -0.67532257 -0.88985527  0.49569136 -0.46274206  0.99644638
  0.25243618  0.3340537  -0.40610438  0.69037801] 
 Updated  value:  2.53862371442035 
 Previous value:  2.5453179965023662 
 x:  [0.         0.         0.         1.         0.29037823 1.
 0.71584307 1.         0.   

Iteration:  119 
 Gradient:   [-0.42100437 -0.7103856  -0.93740933  0.47925069 -0.45760096  1.01734538
  0.26013426  0.32889903 -0.41569025  0.72495949] 
 Updated  value:  2.6218037440511535 
 Previous value:  2.6123783929444575 
 x:  [0.         0.         0.         1.         0.15214684 1.
 0.79332909 1.         0.         1.        ] 

Iteration:  120 
 Gradient:   [-0.44479496 -0.71379944 -0.90969622  0.47962598 -0.44817561  0.98157245
  0.22010421  0.30239094 -0.46787673  0.72050116] 
 Updated  value:  2.630706846624535 
 Previous value:  2.641832333810863 
 x:  [0.         0.         0.         1.         0.14766509 1.
 0.79553013 1.         0.         1.        ] 

Iteration:  121 
 Gradient:   [-0.4358783  -0.69754642 -0.95162761  0.49946693 -0.46188521  1.0116956
  0.25551541  0.30459976 -0.43589293  0.68508996] 
 Updated  value:  2.6238520458196635 
 Previous value:  2.626757197688782 
 x:  [0.         0.         0.         1.         0.14304624 1.
 0.79808529 1.         0. 

Iteration:  149 
 Gradient:   [-0.42286503 -0.70027749 -0.93623144  0.47208842 -0.45503041  0.99903049
  0.2539758   0.32738654 -0.43365698  0.70647052] 
 Updated  value:  2.7018909748468163 
 Previous value:  2.7065233910387385 
 x:  [0.         0.         0.         1.         0.01420678 1.
 0.87023005 1.         0.         1.        ] 

Iteration:  150 
 Gradient:   [-0.41908946 -0.68879027 -0.94650008  0.48783276 -0.45588726  0.99544257
  0.25705503  0.30032907 -0.42407111  0.69071261] 
 Updated  value:  2.7044615251486426 
 Previous value:  2.707714842706639 
 x:  [0.         0.         0.         1.         0.00964791 1.
 0.8728006  1.         0.         1.        ] 

Iteration:  151 
 Gradient:   [-0.43674871 -0.70045157 -0.92612332  0.48389667 -0.47045371  0.99664759
  0.26013426  0.31386459 -0.43211736  0.70389997] 
 Updated  value:  2.7089062943745397 
 Previous value:  2.70617522534986 
 x:  [0.         0.         0.         1.         0.00494337 1.
 0.87540195 1.         0.

Iteration:  173 
 Gradient:   [-0.43143353 -0.69273992 -0.92303052  0.48235705 -0.46359891  0.99628586
  0.25705503  0.30409107 -0.4334829   0.69653648] 
 Updated  value:  2.7277569965879334 
 Previous value:  2.7269001464873246 
 x:  [0.         0.         0.         1.         0.         1.
 0.93195405 1.         0.         1.        ] 

Iteration:  174 
 Gradient:   [-0.42886298 -0.70130842 -0.92560107  0.4815002  -0.45931466  0.99885641
  0.25705503  0.31351642 -0.438624    0.69310908] 
 Updated  value:  2.7277569965879334 
 Previous value:  2.7226158959842808 
 x:  [0.        0.        0.        1.        0.        1.        0.9345246
 1.        0.        1.       ] 

Iteration:  175 
 Gradient:   [-0.43486093 -0.70302212 -0.93159903  0.4849276  -0.46445576  1.00057011
  0.25705503  0.31265957 -0.4317692   0.69482278] 
 Updated  value:  2.7311843969903684 
 Previous value:  2.724329596185498 
 x:  [0.         0.         0.         1.         0.         1.
 0.93709515 1.         0.